<span style="color:red">WIPO PUBLIC</span> 

# Mining Technology Dataset
* *version 1.0 -- AD 2019-07-27*
* *version 1.1 -- AD 2019-07-27* -- adding code for obtaining figure data


This notebook retrieves all the mining technology subset data required for analysis

## Project Outputs

This project creates the following files:

##### raw_data
* mining_first_fam.csv (WIPO families, exclusions not yet removed)
* mining_automation.csv (WIPO families with automation tag)

##### analysis_data
* mining_first_fam.csv (WIPO families, exclusions removed)
* mining_first_fam_sum.csv (summary table for families)
* mining_first_fam_app.csv (family relation table)

##### agg_data
* mining_fam_counts.csv (aggregated mining data by year and origin for quick analysis or RSI calcs)
* first_fam_counts.csv (aggregated comparison data of all technologies)
* RSI_mining_ff.csv (aggregated mining data for RSI calculations)

## Global Variables

In [1]:
# global variables
database_name = "patstat_plus"
print(database_name)

## These tables are from the original mining search
#mining_patents_table = database_name+".mining_tech_min"
#search_table_name = database_name+".mining_patents"
#mining_families = database_name+".mining_first_fam"

# These tables are from the updated 2019 version of the search
mining_patents_table = database_name+".mining2019_min_patents"
search_table_name = database_name+".mining2019_patents"
mining_families = database_name+".mining2019_first_fam"

## Can also consider INPADOC or DOCDB familiy tables
#mining_families = database_name+".mining_inpadoc_fam"
#mining_families = database_name+".mining2019_inpadoc"
#mining_families = database_name+".mining2019_docdb"

print(mining_patents_table)
print(search_table_name)
print(mining_families)

patstat_plus
patstat_plus.mining2019_min_patents
patstat_plus.mining2019_patents
patstat_plus.mining2019_first_fam


##### Import packages

In [2]:
import numpy as np
import pandas as pd
import pyodbc
import time
import os
import csv
import re

##### Create conntection to WIPOstats server

In [ ]:
# global variables: database_name
# This connect works for the WIPO MySQL database, other databases will need alternative database connections
# UID user name deleted for sharing
# PWD password deleted for sharing
con = pyodbc.connect("""DRIVER={MySQL ODBC 8.0 Unicode Driver}; 
                        SERVER=statsql.wipo.int; 
                        DATABASE="""+database_name+"""; 
                        UID = ;
                        PWD = """)

cur = con.cursor()

##### Create a function for importing data from server to dataframes

In [4]:
def get_data(custom_sql):
    SQL = custom_sql
    df_name = pd.read_sql(SQL, con)
    return df_name

##### Download families raw data and remove exclusions 

In [5]:
## download families raw data
#SQL = """SELECT DISTINCT * FROM """+mining_families
#FF_df = get_data(SQL)
#FF_df.drop_duplicates()
#FF_df.to_csv("raw_data/mining_first_fam.csv",header=True,index=False)

In [3]:
# upload family data from csv if already downloaded from server
FF_df = pd.read_csv('raw_data/mining_first_fam.csv')

In [4]:
# apply minimum for category per family
FF_min = FF_df.groupby('fam_id')['min_mining_cat_id'].min()
FF_min = FF_min.to_frame()
FF_min = FF_min.rename(columns={"min_mining_cat_id": "new_min_mining_cat_id"})
FF_min = FF_min.reset_index()
FF_df = pd.merge(FF_df,FF_min, how='left', on='fam_id')
FF_df = FF_df.drop('min_mining_cat_id', axis=1)
FF_df = FF_df.rename(columns={"new_min_mining_cat_id": "min_mining_cat_id"})

# apply floor to get level 1 data
FF_df['min_mining_lvl_id'] = FF_df['min_mining_cat_id'].apply(np.floor)
FF_df.drop_duplicates()

# drop exclusions
FF_df = FF_df.set_index('min_mining_lvl_id')
FF_df = FF_df.drop(0)
FF_df = FF_df.drop(100)
FF_df = FF_df.drop(109)
FF_df = FF_df.reset_index()

FF_df = FF_df.set_index('min_mining_cat_id')
FF_df = FF_df.drop(6.29)
FF_df = FF_df.reset_index()

### Get analysis tables

#### Automation flag

In [8]:
#SQL = """SELECT DISTINCT ffr.fam_id, 'Y' as automation_flag
#         FROM """+search_table_name+""" w
#         INNER JOIN gstat_miscellaneous.family_appln_relation ffr on w.appln_id = ffr.appln_id
#         WHERE mining_cat_id = 10"""
#auto_df = get_data(SQL)
#auto_df = auto_df.drop_duplicates()
#auto_df.to_csv("raw_data/mining_automation.csv",header=True,index=False)
#
#FF_df = pd.merge(FF_df,auto_df, how='left', on='fam_id')
#FF_df['automation_flag'] = FF_df['automation_flag'].fillna(value='N')
#FF_df.drop_duplicates()
#FF_df.to_csv("analysis_data/mining_first_fam.csv",header=True,index=False)

In [5]:
# upload family data from csv if already downloaded from server
FF_df = pd.read_csv('analysis_data/mining_first_fam.csv')

Keep only fam_id for merging with additional tables below

In [6]:
fam_id = FF_df[['fam_id']]

#### Family summary data

In [11]:
#SQL = """SELECT DISTINCT fs.*
#         FROM """+mining_families+""" w
#         INNER JOIN gstat_miscellaneous.family_summary fs using(fam_id)"""
#fam_sum_df = get_data(SQL)
#fam_sum_df = pd.merge(fam_id,fam_sum_df, how='inner', on='fam_id')
#fam_sum_df.drop_duplicates()
#fam_sum_df.to_csv("analysis_data/mining_first_fam_sum.csv",header=True,index=False)

In [7]:
# upload family data from csv if already downloaded from server
fam_sum_df = pd.read_csv('analysis_data/mining_first_fam_sum.csv')

#### Family application data

In [12]:
#SQL = """SELECT DISTINCT far.fam_id, far.appln_id, far.office_code, far.year
#         FROM """+mining_families+""" w
#         INNER JOIN gstat_miscellaneous.family_appln_relation far using(fam_id)"""
#fam_app_df = get_data(SQL)
#fam_app_df = pd.merge(fam_id,fam_app_df, how='inner', on='fam_id')
#fam_app_df.drop_duplicates()
#fam_app_df.to_csv("analysis_data/mining_first_fam_app.csv",header=True,index=False)

In [8]:
# upload family data from csv if already downloaded from server
fam_app_df = pd.read_csv('analysis_data/mining_first_fam_app.csv')

#### Data aggregations for RSI
Aggregate by category, country, year


In [9]:
# create summary table for aggregating
summary_FF_mining = pd.merge(FF_df,fam_sum_df, how='left', on='fam_id')
summary_FF_mining = summary_FF_mining[['fam_id', 'min_mining_lvl_id','origin','year','automation_flag']]
summary_FF_mining.head()

,fam_id,min_mining_lvl_id,origin,year,automation_flag
0,387,5.0,DE,2008,N
1,387,5.0,DE,2008,N
2,387,5.0,DE,2008,N
3,387,5.0,DE,2008,N
4,387,5.0,DE,2008,N


In [90]:
# sum total mining
min_total = summary_FF_mining[['fam_id','origin','year']]
min_total = min_total.drop_duplicates()
min_total = min_total.fillna(0)
min_total = min_total.groupby(['year', 'origin'])['fam_id'].count()
min_total = min_total.to_frame()
min_total = min_total.rename(columns={"fam_id": "total_mining_families"})
min_total.to_csv("agg_data/mining_fam_counts.csv",header=True,index=True)
min_total.head()

total_mining_families
year origin                       
1970 AR                          2
     AT                         36
     AU                        115
     BA                          1
     BE                         85

In [91]:
# sum automation flag
min_auto = summary_FF_mining.loc[summary_FF_mining['automation_flag'] == 'Y']
min_auto = min_auto[['fam_id','origin','year']]
min_auto = min_auto.drop_duplicates()
min_auto = min_auto.fillna(0)
min_auto = min_auto.groupby(['year', 'origin'])['fam_id'].count()
min_auto = min_auto.to_frame()
min_auto = min_auto.rename(columns={"fam_id": "automation families"})
min_total = pd.merge(min_total,min_auto, how='left', left_index=True, right_index=True)
min_total.head()

total_mining_families  automation families
year origin                                            
1970 AR                          2                  NaN
     AT                         36                  1.0
     AU                        115                  NaN
     BA                          1                  NaN
     BE                         85                  NaN

In [ ]:
# sum categories
# create category dictionary
mining_dict = [[1,'blasting families'],
               [2,'environmental families'],
               [3,'processing families'],
               [5,'transport families'],
               [6,'exploration families'],
               [11,'refining families'],
               [12,'metallurgy families'],
               [4,'mining 4'],
               [9,'mining 9']]

summary_FF_mining = summary_FF_mining.fillna(0)

# sum categories
for i,k in mining_dict:
    min_cat = summary_FF_mining.loc[summary_FF_mining['min_mining_lvl_id'] == i]
    min_cat = min_cat[['fam_id','origin','year']]
    min_cat = min_cat.drop_duplicates()
    min_cat = min_cat.groupby(['year', 'origin'])['fam_id'].count()
    min_cat = min_cat.to_frame()
    min_cat = min_cat.rename(columns={"fam_id": k})
    min_total = pd.merge(min_total,min_cat, how='left', left_index=True, right_index=True)
    
min_total = min_total.fillna(0)
min_total['mining operations families'] = min_total['mining 4'] + min_total['mining 9']
min_total = min_total.drop('mining 4', axis=1)
min_total = min_total.drop('mining 9', axis=1)
min_total.to_csv("agg_data/RSI_mining_ff.csv",header=True,index=True)

In [98]:
min_total.head()

total_mining_families  automation families  blasting families  \
year origin                                                                  
1970 AR                          2                  0.0                0.0   
     AT                         36                  1.0                0.0   
     AU                        115                  0.0                2.0   
     BA                          1                  0.0                0.0   
     BE                         85                  0.0                0.0   

             environmental families  processing families  transport families  \
year origin                                                                    
1970 AR                         0.0                  0.0                 0.0   
     AT                         3.0                  1.0                 1.0   
     AU                         8.0                  4.0                 4.0   
     BA                         0.0                  0.0                 0.0   
     BE                         3.0                  2.0                 2.0   

             exploration families  refining families  metallurgy families  \
year origin                                                                 
1970 AR                       1.0                0.0                  0.0   
     AT                       4.0               13.0                  7.0   
     AU                      32.0               37.0                  3.0   
     BA                       0.0                0.0                  0.0   
     BE                       5.0               54.0                  6.0   

             mining operations families  
year origin                              
1970 AR                             1.0  
     AT                             7.0  
     AU                            22.0  
     BA                             1.0  
     BE                             7.0

#### calculate RSI
`log10[[ni/ntotal)/(Ni/Ntotal)]]`
* ni = num tech patents in country i
* ntotal  = num tech patents in total
* Ni = num patents in country i
* Ntotal = num patents in total

want to aggregate by years... this I can't figure out.

In [13]:
# ntotal (mining)

years1 = np.arange(1990,2005)
years2 = np.arange(2005,2016)


min_total_90_04 = min_total

min_total_90_04 = min_total_90_04.reset_index()
min_total_90_04 = min_total_90_04[min_total_90_04['year'].isin(years1)]

ni_90_04 =  min_total_90_04['origin']+min_total_90_04['total_mining_families']

ni_90_04.head()


### Global data for comparison

* data for analysis/first_fam_counts (aggregated by year and country)


In [14]:
# all technologies for RSI calcs
#SQL = """SELECT COUNT(DISTINCT fs.fam_id), origin, year
#        FROM gstat_miscellaneous.family_summary fs 
#        GROUP BY fs.origin, fs.year"""
#all_tech_df = get_data(SQL)
#all_tech_df.to_csv("agg_data/first_fam_counts.csv",header=True,index=False)

In [ ]:
# upload family data from csv if already downloaded from server
all_tech_df = pd.read_csv('agg_data/first_fam_counts.csv')

In [ ]:
all_tech_df.head()

## Graphs and Data featured in the [Working paper](https://www.wipo.int/publications/en/details.jsp?id=4420&plang=EN)

In [19]:
years = np.arange(1990,2016)
years

array([1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000,
       2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
       2012, 2013, 2014, 2015])

#### Number of families between 1990 and 2015
<img src="figures/fig5.jpg" align="left" width="200">


In [ ]:
ff_year = summary_FF_mining[summary_FF_mining.year.isin(years)]
ff_year = ff_year[['fam_id','year']]
ff_year = ff_year.drop_duplicates()
ff_year = ff_year.fillna(0)
ff_year = ff_year.groupby(['year'])['fam_id'].count()
ff_year = ff_year.to_frame()
ff_year = ff_year.rename(columns={"fam_id": "total_mining_families"})
ff_year.sum()

#### Fig 7. Families by priority year
<img src="figures/fig7.jpg" align="left" width="500">

In [ ]:
ff_year.head()

#### Fig 8. Share of families by priority year
<img src="figures/fig8.jpg" align="left" width="500">

In [ ]:
all_ff = all_tech_df[all_tech_df.year.isin(years)]
all_ff = all_ff[['total_WIPO_families','year']]
all_ff = all_ff.drop_duplicates()
all_ff = all_ff.fillna(0)
all_ff = all_ff.groupby(['year'])['total_WIPO_families'].sum()
all_ff = all_ff.to_frame()
all_ff = pd.merge(all_ff,ff_year, how='left', left_index=True, right_index=True)
all_ff['share'] = 100 * all_ff['total_mining_families']/all_ff['total_WIPO_families']
all_ff.head()

#### Fig 9. Families by patent type, by year
<img src="figures/fig9.jpg" align="left" width="500">

#### Table 3. Families by sector between 1990 and 2015 (table)
<img src="figures/tab3.jpg" align="left" width="500">

#### Fig 10. Families by sector between 1990 and 2015, as a share of the whole dataset
<img src="figures/fig10.jpg" align="left" width="500">

#### Fig 11. Families by sector between 1990 and 2015, as a share of the whole dataset, over time
(we did in 5 year increments)

<img src="figures/fig11.jpg" align="left" width="500">

#### Fig 12. Families in mining automation (according to flag), by year and as a share of dataset
<img src="figures/fig12.jpg" align="left" width="500">

#### Fig 13. Families by country
<img src="figures/fig13.jpg" align="left" width="300">

#### Fig 14. Families by top countries of origin, by year
<img src="figures/fig14.jpg" align="left" width="500">

#### Fig 15. Families by country by year
(we did in 5 year increments)

<img src="figures/fig15.jpg" align="left" width="400">

#### Fig 16. RSI by country
(1900-2004 vs 2005-2015)

<img src="figures/fig16.jpg" align="left" width="400">

#### Fig 17. RSI by sector for select countries
(1900-2004 vs 2005-2015)

<img src="figures/fig17.jpg" align="left" width="400">

#### Fig 18. Number of mining patents over the years by type of stakeholder
<img src="figures/fig18.jpg" align="left" width="500">

#### Fig 22. Top 10 filing jurisdictions for mining patents by earliest priority year
<img src="figures/fig22.jpg" align="left" width="500">

#### Fig 23: Origin of application for technology subset: Resident versus non-resident patent filings
<img src="figures/fig23.jpg" align="left" width="500">

##### Close connection to server

In [ ]:
con.close()